In [92]:
# automatically loads the best weights for you
#trainer.test(model)

In [93]:
#trainer.predict(model, datamodule)

In [94]:
# automatically auto-loads the best weights from the previous run

#trainer.test(model)

In [95]:
# load from checkpoint test

#model = MyLightningModule.load_from_checkpoint(PATH)
#trainer = Trainer()
#trainer.test(model, dataloaders=test_dataloader)

In [1]:
import pytorch_lightning as pl
import os
import torch
from torch import nn
from torch import optim
from torch.utils.data import DataLoader, random_split
import torchvision
from torchvision import transforms, models
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from torchvision.datasets import ImageFolder
import torchmetrics
from pytorch_lightning import Trainer
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.callbacks import ModelCheckpoint
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import LearningRateMonitor

In [9]:
class FlowersClassifierDataModule(pl.LightningDataModule):
    def __init__(self, batch_size: int=16, data_dir: str = r"C:\Users\matth\OneDrive\Desktop\school\archive (1)\flower_data\flower_data"):
        super().__init__()
        self.data_dir = data_dir
        self.batch_size = batch_size

        # Augmentation policy for training set
    def setup(self, stage):
        transform_train = transforms.Compose([
              transforms.RandomResizedCrop(size=256),
              transforms.RandomRotation(degrees=15),
              transforms.RandomHorizontalFlip(),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        # Preprocessing steps applied to validation and test set.
        transform_val = transforms.Compose([
              transforms.Resize(size=256),
              transforms.CenterCrop(size=224),
              transforms.ToTensor(),
              transforms.Normalize([0.485, 0.456, 0.406],[0.229, 0.224, 0.225])
        ])
        
        tr_data = ImageFolder(self.data_dir + '\\train', transform_train)
        self.test_dataset = ImageFolder(self.data_dir + '\\valid', transform_val)
        
  
        train_set_size = int(len(tr_data) * 0.95)
        valid_set_size = len(tr_data) - train_set_size

    # split the train set into two
        seed = torch.Generator().manual_seed(42)
        self.train_dataset, self.val_dataset = random_split(tr_data, [train_set_size, valid_set_size], generator=seed)

            
    def train_dataloader(self):
        return DataLoader(self.train_dataset, batch_size=self.batch_size, num_workers=6)

    def val_dataloader(self):
        return DataLoader(self.val_dataset, batch_size=self.batch_size, num_workers=6)
    
    def test_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=6)
    
    def predict_dataloader(self):
        return DataLoader(self.test_dataset, batch_size=self.batch_size, num_workers=6)
        

In [3]:
class FlowersClassifier(pl.LightningModule):
    def __init__(self):
        super().__init__()
        self.criterion =  nn.CrossEntropyLoss()
        self.accuracy = torchmetrics.Accuracy(average='macro', num_classes= 102)
        self.backbone = models.resnet18(pretrained=True)
        self.linear = nn.Linear(1000, 102)
        
    def forward(self, x):
        x = self.backbone(x)
        x = self.linear(x)
        return(x)
    
    def training_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy(y_hat, y)
        self.log("mytrain_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("mytrain_acc", acc, on_epoch=True, prog_bar=True, logger=True)
        return {'loss': loss,'accuracy': acc,}
        
    def validation_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy(y_hat, y)
        self.log("val_loss", loss, on_epoch=True, prog_bar=True, logger=True)
        self.log("val_accuracy", acc, on_epoch=True, prog_bar=True, logger=True)
        return {'val_loss': loss,'val_accuracy': acc, "pred": y_hat.detach(), "labels": y}
    
    def test_step(self, batch, batch_idx):
        x, y = batch
        y_hat = self(x)
        loss = self.criterion(y_hat, y)
        acc = self.accuracy(y_hat, y)
        metrics = {"test_loss": loss, "test_accuracy": acc}
        self.log_dict(metrics)
        return metrics

    def predict_step(self, batch, batch_idx, dataloader_idx=0):
        x, y = batch
        y_hat = self(x)
        return y_hat
    
    def validation_epoch_end(self, outputs):
        val_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        log = {"avg_val_loss": val_loss}
        self.log("avg_val_loss", val_loss, on_epoch=True, prog_bar=True, logger=True)
        return {"log": log, "val_loss": val_loss}

    def configure_optimizers(self):
        optimizer = optim.SGD(model.parameters(), lr=1e-2)
        return optimizer

In [4]:
model = FlowersClassifier()
datamodule = FlowersClassifierDataModule()

logger = TensorBoardLogger("lightning_logs", name="my_model")
# saves top-K checkpoints based on "val_loss" metric
checkpoint_callback = ModelCheckpoint(
    save_top_k=1,
    monitor="val_loss",
    mode="min",
    dirpath=r"\Users\matth\OneDrive\Desktop\school\archive (1)")

C:\Users\matth\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and will be removed in 0.15, please use 'weights' instead.
  warnings.warn(
C:\Users\matth\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [5]:
trainer = pl.Trainer(accelerator="gpu", devices=1, max_epochs=20, log_every_n_steps=16, callbacks=[EarlyStopping(monitor="mytrain_loss", mode="min"), checkpoint_callback])

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [6]:
trainer.fit(model, datamodule)

C:\Users\matth\anaconda3\lib\site-packages\pytorch_lightning\callbacks\model_checkpoint.py:616: UserWarning: Checkpoint directory C:\Users\matth\OneDrive\Desktop\school\archive (1) exists and is not empty.
  rank_zero_warn(f"Checkpoint directory {dirpath} exists and is not empty.")
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type             | Params
-----------------------------------------------
0 | criterion | CrossEntropyLoss | 0     
1 | accuracy  | Accuracy         | 0     
2 | backbone  | ResNet           | 11.7 M
3 | linear    | Linear           | 102 K 
-----------------------------------------------
11.8 M    Trainable params
0         Non-trainable params
11.8 M    Total params
47.166    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

...

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=20` reached.


In [7]:
# automatically auto-loads the best weights from the previous run

dm = FlowersClassifierDataModule()

trainer.test(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       Test metric             DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      test_accuracy         0.8267789483070374
        test_loss           0.1669778823852539
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


[{'test_loss': 0.1669778823852539, 'test_accuracy': 0.8267789483070374}]

In [10]:
#outputs tensors

dm = FlowersClassifierDataModule()

predictions = trainer.predict(model, datamodule=dm)

LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Predicting: 389it [00:00, ?it/s]

[tensor([[ 19.9797,  -2.5054,  -2.7752,  ...,   4.8495,   2.2800,  -7.3665],
         [ 26.4249,  -4.9671,  -3.9212,  ...,   4.4823,   5.4735, -10.6042],
         [ 23.0775,  -2.1155,  -1.6833,  ...,   6.1773,   2.7814, -10.2371],
         ...,
         [ -5.0666,   3.5981,  21.1483,  ...,   2.9519,  -6.1566,   4.7181],
         [-10.0946,   1.7630,  22.5440,  ...,  -0.3415,  -2.9383,   6.1123],
         [ -4.4721,   3.8064,  23.7839,  ...,   1.8412,  -2.4666,   2.1303]]),
 tensor([[-9.5627,  8.9698, 18.8213,  ...,  2.9813, -4.9705,  1.4096],
         [-5.7121,  3.9437, 21.4284,  ...,  0.4167, -8.4330,  4.4269],
         [-4.9498, -9.1242,  3.9755,  ..., 10.0731, -5.6682,  5.6051],
         ...,
         [-2.5744, -0.3817,  0.4739,  ..., -1.7861, -5.6131, -5.2480],
         [-2.3096, -0.3832, -1.4977,  ..., -0.6405,  1.9082, -0.9869],
         [-7.6021, -3.4779,  0.1527,  ..., -1.3433, -3.7863,  4.4395]]),
 tensor([[ 0.6186, -2.4110, -3.2034,  ..., -0.5869, -1.2816, -3.5824],
         

In [57]:
#Can't yet implement the dataloader into the predictior.

dm = FlowersClassifierDataModule()

predictions = trainer.predict(model, datamodule=dm)

probs = nn.Softmax() # since the outputs are logits we use the softmax function to obtain probabilities
def get_score(y_hat,y):
    probabilities = []
    accuracy = 0
    for x in y_hat:
        prob = probs(x)
        top_p, top_class = prob.topk(1, dim = -1)
        probabilities.append(float(top_class))
    i = 0
    for x in y:
        if int(x) == int(probabilities[i]):
            accuracy = accuracy + 1
        i = i +1
    score = accuracy/len(probabilities)
    return score # returns accuracy
get_score(predictions,"/test set")

TypeError: Unwrapping the module did not yield a `LightningModule`, got <class 'type'> instead.

In [61]:
%load_ext tensorboard 
%tensorboard --logdir lightning_logs/